In [1]:

from ctgan import TVAE
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import torch
import pandas as pd
import numpy as np
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [2]:
import tensorflow as tf
print(torch.cuda.is_available())

True


In [3]:
real_data = pd.read_csv("cicids2018.csv")

In [4]:
scaler = StandardScaler()
label = LabelEncoder()
# data[data.columns[:-1]] = scaler.fit_transform(data[data.columns[:-1]])
real_data[real_data.columns[:-1]] = np.array(real_data[real_data.columns[:-1]], dtype=float)
real_data.replace([np.inf, -np.inf], np.nan, inplace=True)
#data.iloc[:, -1] = label.fit_transform(data.iloc[:, -1])
real_data.dropna(inplace=True)
real_data.head(3).T

,0,1,2
Dst Port,80.0,80.0,80.0
Protocol,6.0,6.0,6.0
Flow Duration,5007793.0,8938396.0,52700.0
Tot Fwd Pkts,4.0,2.0,5.0
Tot Bwd Pkts,4.0,0.0,3.0
...,...,...,...
Idle Mean,0.0,8938396.0,0.0
Idle Std,0.0,0.0,0.0
Idle Max,0.0,8938396.0,0.0
Idle Min,0.0,8938396.0,0.0


In [ ]:

def get_discrete_columns(dataframe, max_unique_values=80):
    """
    Hàm để xác định các cột rời rạc (discrete), bao gồm các cột có số lượng giá trị unique ít hơn hoặc bằng max_unique_values.
    Ngoài ra, có thể chuyển đổi kiểu dữ liệu của các cột rời rạc sang string.
    
    - dataframe: DataFrame chứa dữ liệu
    - max_unique_values: Số lượng giá trị unique tối đa để coi là discrete (mặc định là 80)
    - convert_to_string: Nếu True, chuyển kiểu dữ liệu của các cột rời rạc sang string.
    
    Trả về:
    - discrete_columns: Danh sách tên các cột rời rạc.
    - discrete_indices: Danh sách chỉ số (index) của các cột rời rạc.
    """
    discrete_columns = []
    discrete_indices = []
    
    for col in dataframe.columns:
        unique_values = dataframe[col].nunique()
        print(f"Column '{col}' - Number of unique values: {unique_values}")
        # Chọn các cột có kiểu dữ liệu số và số lượng giá trị unique nhỏ hơn hoặc bằng max_unique_values
        if dataframe[col].dtype in ['float64', 'int64'] and unique_values <= max_unique_values:
            discrete_columns.append(col)
            discrete_indices.append(dataframe.columns.get_loc(col))
            
            dataframe[col] = dataframe[col].astype(str)
            print(f"Column '{col}' converted to string.")
    
    return discrete_columns, discrete_indices 


# Ví dụ với dataset của bạn
discrete_columns, discrete_indices = get_discrete_columns(real_data)
discrete_columns.append("Target")
print("Discrete Columns:", discrete_columns)
print("Indices of Discrete Columns:", discrete_indices)

Column 'Dst Port' - Number of unique values: 37447
Column 'Protocol' - Number of unique values: 3
Column 'Protocol' converted to string.
Column 'Flow Duration' - Number of unique values: 472240
Column 'Tot Fwd Pkts' - Number of unique values: 704
Column 'Tot Bwd Pkts' - Number of unique values: 942
Column 'TotLen Fwd Pkts' - Number of unique values: 6486
Column 'TotLen Bwd Pkts' - Number of unique values: 19272
Column 'Fwd Pkt Len Max' - Number of unique values: 1453
Column 'Fwd Pkt Len Min' - Number of unique values: 171
Column 'Fwd Pkt Len Mean' - Number of unique values: 21898
Column 'Fwd Pkt Len Std' - Number of unique values: 39819
Column 'Bwd Pkt Len Max' - Number of unique values: 1240
Column 'Bwd Pkt Len Min' - Number of unique values: 317
Column 'Bwd Pkt Len Mean' - Number of unique values: 33207
Column 'Bwd Pkt Len Std' - Number of unique values: 41211
Column 'Flow Byts/s' - Number of unique values: 512726
Column 'Flow Pkts/s' - Number of unique values: 493873
Column 'Flow IA

In [6]:
tvae = TVAE()

In [7]:
#ctgan.load_model("2018\ctgan_model_epoch_10.pth")
tvae.fit(real_data, discrete_columns)

c:\Final Project in Uni\CTGAN\ctgan\synthesizers\ctgan.py:687: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=device)


Model loaded from 2018\ctgan_model_epoch_10.pth


In [8]:
def get_unique_labels(dataframe, label_column):
    """
    Trả về mảng các giá trị nhãn duy nhất từ một cột trong tập dữ liệu.

    Args:
        dataframe (pd.DataFrame): Tập dữ liệu đầu vào.
        label_column (str): Tên cột chứa nhãn.

    Returns:
        list: Danh sách các nhãn duy nhất.
    """
    if label_column not in dataframe.columns:
        raise ValueError(f"Cột '{label_column}' không tồn tại trong tập dữ liệu.")
    
    unique_labels = dataframe[label_column].unique()
    return unique_labels.tolist()

In [9]:
unique_labels = get_unique_labels(real_data, 'Target')
unique_labels

['DoS attacks-GoldenEye',
 'Infilteration',
 'SSH-Bruteforce',
 'Bot',
 'DoS attacks-Hulk',
 'DDoS attacks-LOIC-HTTP',
 'DDOS attack-HOIC',
 'Benign']

In [11]:
synthetic_data = tvae.sample(len(real_data))

In [ ]:
synthetic_data.to_csv("2018/tvae_10e_2018_80pt.csv", index=False)